In [1]:
import itertools
import numpy as np
from gurobipy import Model, GRB

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    attributes = ["A", "B", "C", "D", "E", "F", "G", "H"]
    relations = [["A","B","C","D"], ["A","B","E","F"], ["C","D","E","F"], ["A","B","G","H"], ["C","D","G","H"]]
    
    #attributes = ["x", "y", "z"]
    #relations = [["x", "y"], ["y", "z"], ["z", "x"]]
    
    #attributes = ["A", "B", "X", "Y", "C"]
    #relations = [["A","B","X","Y", "C"], ["X","Y"], ["A","X"], ["A","Y"], ["B","X"], ["B", "Y"], ["C"]]
    
    #attributes = ["A", "B", "C", "D", "E"]
    #relations = [["A", "B"], ["B", "C"], ["C", "D"], ["D", "E"]]
    
    #attributes = ["x", "y", "z"]
    #relations = [["x", "y"], ["y", "z"]]
    
    #attributes = ["x", "y", "z", "u"]
    #relations = [["x", "y"], ["y", "z"], ["z", "u"]]
    
    variables = {}
    for i in range(1, len(attributes) + 1):
        for rho_set in itertools.combinations(attributes, i):
            if frozenset(rho_set) not in variables:
                rho = "".join(rho_set)
                name = f"D({rho}||I/|{rho}|)"
                x = model.addVar(lb=0, name=name)
                variables[frozenset(rho_set)] = x
    
    objective = len(attributes) - variables[frozenset(attributes)] 
    
    for relation in relations:
        model.addConstr(variables[frozenset(relation)] >= len(relation) - 1, f"rel_{"".join(relation)}")
        #objective += variables[frozenset(relation)]
    
    model.setObjective(objective, GRB.MAXIMIZE)
        
    #fds = [[["E","F","G","H"], ["A","B","C","D"]], [["A","B","C","E"], ["D","F","G","H"]], [["B","C","D","G"], ["A","E","F","H"]]]
    #candidate_keys = [[["E","F","G","H"], ["A","B","C","D", "E","F","G","H"]], [["A","B","C","E"], ["D","F","G","H", "A","B","C","E"]], [["B","C","D","G"], ["A","E","F","H", "B","C","D","G"]]]
    #fds = fds + candidate_keys
    #for Y, x in fds:
    #    Yx = Y + x
    #    model.addConstr(variables[(frozenset(Y), frozenset(Yx))] == 0, "fd_" + "".join(Y) + "->" + "".join(x))
    
    # Monotonicty of relative entropy
    for rho in variables:
        if len(rho) > 1:
            for traced_out_element in rho:
                rho_prime = rho - frozenset([traced_out_element])
                model.addConstr(variables[rho] >= variables[rho_prime], f"monotonicity_{"".join(rho_set)}_{"".join(rho_prime)}")
    
    
    model.write("relative_entropy_program1.lp")
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
        return model.objVal
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")
    
result = solve_linear_program()
print(f"Result: {result}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1021 rows, 255 columns and 2037 nonzeros
Model fingerprint: 0x4129996b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 1021 rows and 255 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.000000000e+00
Optimal solution found!
Objective value = 5.0
Result: 5.0
